In [ ]:
%pylab inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import argparse
from scipy import interpolate
import tqdm
import nmmn.lsd, nmmn.misc
import os.path
import pandas as pd
import astropy.visualization

In [ ]:
# Constants, definitions and units

# many are related to grmonty, but we can set a black hole mass if we want to have a "feeling" for the physical values

global MP, ME, CL, GNEWT, KBOL, SIGMA_THOMSON, MSUN, LSUN, YEAR, MBH
global TPTE_DISK, TPTE_JET, THETAE_MAX
global M_unit, L_unit, T_unit, RHO_unit, U_unit, B_unit, Ne_unit

# all constants in cgs units
ME = 9.1093826e-28 # electron mass
MP = 1.67262171e-24 # proton mass
CL = 2.99792458e10 # speed of light
GNEWT = 6.6742e-8 # gravitational constant
KBOL = 1.3806505e-16 # Boltzmann constant
SIGMA_THOMSON = 0.665245873e-24 # Thomson cross-section
MSUN = 1.989e33 # solar mass
LSUN = 3.827e33 # solar luminosity
YEAR = 31536000 # seconds in a year

# temperature and beta-prescription (Mościbrodzka 2016)
TPTE_DISK = 20. # R_high
TPTE_JET = 1. # R_low
THETAE_MAX = 1000.
TP_OVER_TE = 100.0

sgra=1
m87=0
blazar=0
# grmonty units and BH mass
if (sgra):
    MBH = 4.5e6 * MSUN
    M_unit = 1.0e19
elif (m87):
    MBH = 6.2e9 * MSUN
    M_unit = 1.0e29

L_unit = GNEWT * MBH / (CL * CL)
T_unit = L_unit / CL
RHO_unit = M_unit / (L_unit*L_unit*L_unit)
U_unit = RHO_unit * CL * CL
B_unit = CL * np.sqrt(4. * np.pi * RHO_unit)
Ne_unit = RHO_unit / (MP + ME)

#MBH = 5.0e9 * MSUN
#M_unit = 2.0*10e11

In [ ]:
%run -i harm_script2.py

In [ ]:
rg("gdump")

In [ ]:
dumpnumber = 300
dumpname = "dump"+str(dumpnumber)

rd(dumpname)

In [ ]:
N1, N2, N3 = nx, ny, nz

sigma = bsq/rho
sigmaphi = bu[3]*bd[3]/(rho)

betapl = 2.0*pg/bsq

bphi = bu[3]*B_unit
Bphi = B[3]*B_unit
#jcurr = np.nan_to_num(jcurr)
jdotu = mdot(ju,ud)
Jsq = jsq + jdotu*jdotu
gJsq = gdet*Jsq
JJJ = sqrt(Jsq)

EF = rho + gam*ug
EE = bsq + EF
va2 = bsq/EE
va = np.sqrt(va2)

enth=1+ug*gam/rho
unb=enth*ud[0]
isunbound=(-unb>1.02)

sigma2d = sigma[:,:,0]
r2d = (r*np.sin(h))[:,:,0]
z2d = (r*np.cos(h))[:,:,0]

Get current sheet files, in case we want to plot them

In [ ]:
Jfile_j = "/work/gustavo/gyst/jcs_files_jet/"+dumpname+"_jcs"
Jfilechar_j = "/work/gustavo/gyst/jcs_files_jet/"+dumpname+"_jcs_char"
Jfile_d = "/work/gustavo/gyst/jcs_files_disc/"+dumpname+"_jcs"
Jfilechar_d = "/work/gustavo/gyst/jcs_files_disc/"+dumpname+"_jcs_char"

f1_j = open(Jfile_j, "rb")
dtype = np.float64
body_j = np.fromfile(f1_j,dtype=dtype,count=-1)
JJ_cs_j = body_j.view().reshape((int(N1),int(N2),int(N3)), order='C')

f2_j = open(Jfilechar_j, "rb")
dtype = np.float64
body_j = np.fromfile(f2_j,dtype=dtype,count=-1)
JJ_cs_char_j = body_j.view().reshape((int(N1),int(N2),int(N3)), order='C')

f1_d = open(Jfile_d, "rb")
dtype = np.float64
body_d = np.fromfile(f1_d,dtype=dtype,count=-1)
JJ_cs_d = body_d.view().reshape((int(N1),int(N2),int(N3)), order='C')

f2_d = open(Jfilechar_d, "rb")
dtype = np.float64
body_d = np.fromfile(f2_d,dtype=dtype,count=-1)
JJ_cs_char_d = body_d.view().reshape((int(N1),int(N2),int(N3)), order='C')

JJ_cs_char_ind_j = numpy.zeros(shape=(N1,N2,N3))
JJ_cs_char_ind_d = numpy.zeros(shape=(N1,N2,N3))
for i in range(N1):
    for j in range(N2):
        for k in range(N3):
            if JJ_cs_char_j[i][j][k] > 0:
                JJ_cs_char_ind_j[i][j][k] = 1
            if JJ_cs_char_d[i][j][k] > 0:
                JJ_cs_char_ind_d[i][j][k] = 1
                
#Jdata = np.loadtxt(Jfile)
#JJ_cs = np.reshape(Jdata, (int(N1), int(N2), int(N3)))
vasites_j=[]
rx_j = []
ry_j = []
vasites_d=[]
rx_d = []
ry_d = []
for i in range(525):
    for j in range(N2):
        for k in range(N3):
            if (JJ_cs_char_ind_j[i][j][k] == 1):
                rx_j.append(r[i][j][k] * np.sin(h[i][j][k]))
                ry_j.append(r[i][j][k] * np.cos(h[i][j][k]))
                vasites_j.append(va[i][j][k])
            if (JJ_cs_char_ind_d[i][j][k] == 1):
                rx_d.append(r[i][j][k] * np.sin(h[i][j][k]))
                ry_d.append(r[i][j][k] * np.cos(h[i][j][k]))
                vasites_d.append(va[i][j][k])

In [ ]:
mirror = 1
if(mirror):
    xmax = 100
    ymax = xmax/2
    step = xmax/4
else:
    xmax = 125
    ymax = 250
    step = 25

hfont = {'fontname':'Helvetica'}
aphi=psicalc()
amax = aphi.max()
cs, cb = plco(np.log10(rho),levels = np.linspace(-8, 0, 100),isfilled=1,k=0,xy=1,xmax=xmax,ymax=ymax,dobh=1,cb=1,extend="both",pretty=1)
#cs, cb = plco(bu[3]*B_unit, levels = np.linspace(-0.1, 0.1, 100), isfilled=1, k=0, xy=1, xmax=80, ymax=40, dobh=1, cb=1, extend = "both", pretty=1, cmap = 'RdBu')
#plt.contour(r2d, z2d, sigma2d, levels=[1], colors=('black'), linestyles=('-.'), linewidths=(2))
ax = plt.gca()
ax.set_xlabel(r"$R\ [r_g]$",fontsize=20,labelpad=-5, **hfont)
ax.set_ylabel(r"$z\ [r_g]$",fontsize=20,labelpad=-5, **hfont)
cb.ax.set_xlabel(r"$\log\;\rho$",fontsize=20,ha="left")
cb.set_ticks([-8,-7,-6,-5,-4,-3,-2,-1,0])
if(mirror):
    plt.xticks(np.arange(0, xmax+1, step=step), **hfont) #GS
    plt.yticks(np.arange(-ymax, ymax+1, step=step), **hfont) #GS
    plt.xlim(0,xmax+1)
    plt.ylim(-ymax,ymax+1)
else:
    plt.xticks(np.arange(0, xmax+1, step=step), **hfont) #GS
    plt.yticks(np.arange(0, ymax+1, step=step), **hfont) #GS
    plt.xlim(0,xmax+1)
    plt.ylim(0,ymax+1)
circle1 = plt.Circle((0, 0), Rin, color='black') # GS
fig = plt.gcf() # GS
ax2 = fig.gca() # GS
ax2.add_artist(circle1) # GS
plc(aphi,levels=np.linspace(-amax,amax,20)[1:-1],colors="black",linewidths=0.5,xy=-1)
plt.title(r"$t=%.4g$"%np.round(t)+r"$\;r_g/c$", **hfont); 
plt.tight_layout() # GS
plt.draw()
#plt.savefig("rho.png", dpi = 300)